In [1]:
from glob import glob
import numpy as np
import sys, os, h5py, time, errno
import GPUtil
import MDAnalysis as mda
from sklearn.cluster import DBSCAN

from CVAE import CVAE 
from utils import start_rabbit, start_worker, start_flower_monitor, read_h5py_file, cm_to_cvae, job_on_gpu
from utils import find_frame, write_pdb_frame, make_dir_p, outliers_from_cvae, job_list
from utils import omm_job, cvae_job 

Using TensorFlow backend.


In [2]:
top_file = None
pdb_file = os.path.abspath('./pdb/100-fs-peptide-400K.pdb')

In [3]:
cm_files = sorted(glob('omm*/*_cm.h5')) 

cm_data_lists = [read_h5py_file(cm_file) for cm_file in cm_files] 

In [4]:
cm_files

['omm_run_1543441282/output_cm.h5',
 'omm_run_1543553699/output_cm.h5',
 'omm_run_1543553701/output_cm.h5',
 'omm_run_1543553703/output_cm.h5',
 'omm_run_1543553705/output_cm.h5',
 'omm_run_1543553707/output_cm.h5',
 'omm_run_1543553709/output_cm.h5',
 'omm_run_1543553711/output_cm.h5',
 'omm_run_1543553713/output_cm.h5',
 'omm_run_1543553715/output_cm.h5',
 'omm_run_1543553717/output_cm.h5',
 'omm_run_1543553719/output_cm.h5',
 'omm_run_1543553721/output_cm.h5',
 'omm_run_1543553723/output_cm.h5',
 'omm_run_1543553725/output_cm.h5',
 'omm_run_1543553727/output_cm.h5',
 'omm_run_1543553729/output_cm.h5']

# All contact to h5

In [5]:
train_data_length = [ cm_data.shape[1] for cm_data in cm_data_lists]

omm_log = os.path.join('./scheduler_logs/openmm_log.txt') 

log = open(omm_log, 'w') 

for i, n_frame in enumerate(train_data_length): 
    log.writelines("{} {}\n".format(cm_files[i], n_frame))    
log.close()

In [6]:
cvae_input = cm_to_cvae(cm_data_lists)

cvae_input_file = os.path.abspath('./cvae_input/cvae_input.h5')
cvae_input_save = h5py.File(cvae_input_file, 'w')
cvae_input_save.create_dataset('contact_maps', data=cvae_input)
cvae_input_save.close() 

In [7]:
cvae_input.shape

(28909, 22, 22, 1)

# CVAE

# Identifier base on CVAE result

In [11]:
model_weights = glob('cvae_model_*_1543*/cvae_weight.h5') 
print model_weights[0]
model_weights[0][11]

cvae_model_3_1543553853/cvae_weight.h5


'3'

In [16]:
outlier_list = []
for model_weight in model_weights: 
    outliers = outliers_from_cvae(model_weight, cvae_input, hyper_dim=int(model_weight[11]), eps=0.35) 
    outlier_list.append(np.squeeze(outliers))

In [45]:
type(np.array(outliers))

numpy.ndarray

In [70]:
outlier_list = []
for model_weight in model_weights: 
    for eps in np.arange(0.35, 1.0, 0.05): 
        outliers = np.squeeze(outliers_from_cvae(model_weight, cvae_input, hyper_dim=int(model_weight[11]), eps=eps))
        n_outlier = len(outliers)
        print 'dimension = {0}, eps = {1}, number of outlier found: {2}'.format(
            model_weight[11], eps, n_outlier)
        if n_outlier <= 50: 
            print 'Frame number of outliers:'
            print outliers
            outlier_list.append(outliers)
            print 'next dimenstion '
            break

dimension = 3, eps = 0.35, number of outlier found: 89
dimension = 3, eps = 0.4, number of outlier found: 42
Frame number of outliers:
[  486   609   929  3669  4611  5073  5350  5351  5353  5354  5355  5839
  5840  5841  5921  6695  6715  6717  6718  6719  6721  7084  7114  7499
  7667  7942  8189  8501  8527  8528  9090  9971 11009 11316 11403 20161
 20173 20203 20485 20568 20812 20829]
next dimenstion 
dimension = 4, eps = 0.35, number of outlier found: 957
dimension = 4, eps = 0.4, number of outlier found: 449
dimension = 4, eps = 0.45, number of outlier found: 212
dimension = 4, eps = 0.5, number of outlier found: 98
dimension = 4, eps = 0.55, number of outlier found: 43
Frame number of outliers:
[  738   945   991  1698  2002  2024  2152  2157  2326  2328  3608  3781
  4558  4567  4786  5073  5353  5354  5355  5373  5374  5381  5446  5462
  6119  6374  6717  6718  7277  7537  8308  9812  9884 14835 20186 20195
 20208 20454 20502 20603 20624 20626 20631]
next dimenstion 
dimension

In [71]:
(np.hstack(outlier_list)).shape

(131,)

In [72]:
outlier_list_uni, outlier_count = np.unique(np.hstack(outlier_list), return_counts=True)

In [86]:
outlier_list_ulti = outlier_list_uni[np.where(outlier_count > 1)]

In [77]:
print('Writing pdb files') 
# write the pdb according the outlier indices
traj_info = open('./scheduler_logs/openmm_log.txt', 'r').read().split()

traj_dict = dict(zip(traj_info[::2], np.array(traj_info[1::2]).astype(int)))

traj_dict

Writing pdb files


{'omm_run_1543441282/output_cm.h5': 26017,
 'omm_run_1543553699/output_cm.h5': 58,
 'omm_run_1543553701/output_cm.h5': 60,
 'omm_run_1543553703/output_cm.h5': 60,
 'omm_run_1543553705/output_cm.h5': 62,
 'omm_run_1543553707/output_cm.h5': 225,
 'omm_run_1543553709/output_cm.h5': 222,
 'omm_run_1543553711/output_cm.h5': 221,
 'omm_run_1543553713/output_cm.h5': 226,
 'omm_run_1543553715/output_cm.h5': 222,
 'omm_run_1543553717/output_cm.h5': 222,
 'omm_run_1543553719/output_cm.h5': 223,
 'omm_run_1543553721/output_cm.h5': 223,
 'omm_run_1543553723/output_cm.h5': 219,
 'omm_run_1543553725/output_cm.h5': 217,
 'omm_run_1543553727/output_cm.h5': 216,
 'omm_run_1543553729/output_cm.h5': 216}

In [87]:
outliers_pdb = os.path.abspath('./outlier_pdbs')
make_dir_p(outliers_pdb)

for outlier in outlier_list_ulti: 
    traj_file, frame_number = find_frame(traj_dict, outlier) 
    outlier_pdb_file = os.path.join(outliers_pdb, '%d_%s_%d.pdb' % (outlier, traj_file[:18], frame_number))
    outlier_pdb = write_pdb_frame(traj_file, pdb_file, frame_number, outlier_pdb_file) 